# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install varname

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import datetime

from varname import nameof
import time

import numpy as np

# Functions for Calculating Technical Indicators

In [4]:
#function to get the stock name

def variable_to_str(df): 
    variable_name = list(df.keys())[1] 
    return variable_name[6:]

In [5]:
#Moving Average Convergence Divergence (MACD)

#https://aleksandarhaber.com/macd-moving-average-convergence-divergence-of-stock-price-time-series-in-pandas-and-python/


def MACD(data):
    
    short_period = 12
    long_period  = 26
    signal_period = 9

    stock_name = variable_to_str(df=data)
    
    data["ewm_short"] = data["Price_"+stock_name].ewm(span=short_period, adjust=False).mean()
    data["ewm_long"] = data["Price_"+stock_name].ewm(span=long_period, adjust=False).mean()
    data["MACD"] = data["ewm_short"] - data["ewm_long"]
    data["signal_MACD"] = data["MACD"].ewm(span=signal_period, adjust=False).mean()
    data["bars"] = data["MACD"] - data["signal_MACD"]
    
    data.drop(["ewm_short","ewm_long","signal_MACD","bars"],axis=1,inplace=True)
    
    data.rename(columns={"MACD":"MACD_"+stock_name},inplace=True)
    
    

In [6]:
#Relative Strength Index (RSI)

#https://www.youtube.com/watch?v=2H2Gd7zYVA8

def RSI(data):
    
    stock_name = variable_to_str(df=data)
    
    data["delta"] = data["Price_"+stock_name].diff()
    data["up"] = data["delta"].clip(lower=0)
    data["down"] = -1 * data["delta"].clip(upper=0)
    
    data["ema_up"] = data["up"].ewm(com=13, adjust=False).mean()
    data["ema_down"] = data["down"].ewm(com=13, adjust=False).mean()
    
    data["rs"] = data["ema_up"] / data["ema_down"]
    data["RSI"] = 100 - (100 / (1 + data["rs"]))
    
    data.drop(["delta","up","down","ema_up","ema_down","rs"], axis=1, inplace=True)
    data.rename(columns = {"RSI":"RSI_"+stock_name}, inplace=True)
    

In [7]:
#Commodity Channel Index (CCI)

#https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/cci
#https://gist.github.com/quantra-go-algo/1b37bfb74d69148f0dfbdb5a2c7bdb25

#TP = Typical Price
#sma = simple moving average
#mad = mean deviation

def CCI(data, number_of_days):
    
    stock_name = variable_to_str(df=data)
    
    data["TP"] = (data["High_"+stock_name] + data["Low_"+stock_name] + data["Price_"+stock_name])/3
    data['sma'] = data['TP'].rolling(number_of_days).mean()
    data['mad'] = data['TP'].rolling(number_of_days).apply(lambda x: pd.Series(x).mad())
    data['CCI'] = (data['TP'] - data['sma']) / (0.015 * data['mad']) 
    
    data.drop(["TP","sma", "mad"], axis=1, inplace=True)
    data.rename(columns = {"CCI":"CCI_"+stock_name}, inplace=True)


In [8]:
# Average Directional Index (ADX)

#https://medium.com/codex/algorithmic-trading-with-average-directional-index-in-python-2b5a20ecf06a

def calculateValue(df, column, index):
    previous_value = df.loc[index-1, column]
    current_value = df.loc[index, column]
    return current_value, previous_value

def calculateDM(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    
    dm_positive = current_high - previous_high
    dm_negative = current_low - previous_low
    
    if dm_positive > dm_negative:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

    elif dm_positive < dm_negative:
        if dm_negative < 0:
            dm_negative = 0.00
        dm_positive = 0.00
        return dm_positive, dm_negative
    
    else:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

def calculateTR(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    current_close, previous_close = calculateValue(df, "Price_"+stock_name, index)
    ranges = [current_high - current_low, abs(current_high - previous_close), abs(current_low - previous_close)]
    TR = max(ranges)
    return TR

def calculate_first_14(df, index, column):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, column]
    return result

def calculate_subsequent_14(df, index, column):
    return(df.loc[index-1, column+'14'] - (df.loc[index-1, column+'14']/14) + df.loc[index, column])


def calculate_first_adx(df, index):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, 'DX']
    return(result/14)

def calculate_adx(df, index):
    return(round(((df.loc[index-1, 'ADX']*13) + df.loc[index, 'DX'])/14, 2))

def ADX(data):
    
    stock_name = variable_to_str(df=data)
    
    for i in range(1, len(data)):
        dm_pos, dm_neg = calculateDM(data, i, stock_name)
        TR = calculateTR(data, i, stock_name)
        data.loc[i, '+DM'] = dm_pos
        data.loc[i, '-DM'] = dm_neg
        data.loc[i, 'TR'] = TR

        if data.TR.count() == 14:
            data.loc[i, 'TR14'] = calculate_first_14(data, i, 'TR')
            data.loc[i, '+DM14'] = calculate_first_14(data, i, '+DM')
            data.loc[i, '-DM14'] = calculate_first_14(data, i, '-DM')

        elif data.TR.count() >= 14:
            data.loc[i, 'TR14'] = round(calculate_subsequent_14(data, i, 'TR'),2)
            data.loc[i, '+DM14'] = round(calculate_subsequent_14(data, i, '+DM'), 2)
            data.loc[i, '-DM14'] = round(calculate_subsequent_14(data, i, '-DM'), 2)

        if 'TR14' in data.columns:
            data.loc[i, '+DI'] = round((data.loc[i, '+DM14'] / data.loc[i, 'TR14'])*100, 2)
            data.loc[i, '-DI'] = round((data.loc[i, '-DM14'] / data.loc[i, 'TR14'])*100, 2)

            data.loc[i, 'DX'] = round((abs(data.loc[i, '+DI'] - data.loc[i, '-DI'])/abs(data.loc[i, '+DI'] + data.loc[i, '-DI']) )*100 , 2)

        if 'DX' in data.columns:
            if data.DX.count() == 14:
                data.loc[i, 'ADX'] = calculate_first_adx(data, i)

            elif data.DX.count() >= 14:
                data.loc[i, 'ADX'] = calculate_adx(data, i)
    
    data.drop(["+DM","-DM","TR","TR14","+DM14","-DM14","+DI","-DI","DX"], axis=1, inplace=True)
    data.rename(columns={"ADX":"ADX_"+stock_name},inplace=True)




# BXPH - Beximco Pharmaceuticals Ltd

In [25]:
BXPH = pd.read_csv("/content/drive/MyDrive/stock project/data/BXPH.csv")
len_bxph = len(BXPH)
print(len_bxph)
BXPH.head()

1948


,Date,Price,Open,High,Low,Vol.,Change %
0,10/30/2022,153.2,156.3,156.3,147.8,362.01K,-2.05%
1,10/27/2022,156.4,159.0,160.9,156.0,185.86K,-1.01%
2,10/26/2022,158.0,160.0,161.1,153.1,354.48K,-1.37%
3,10/25/2022,160.2,162.0,164.3,159.5,299.41K,0.00%
4,10/24/2022,160.2,170.0,170.0,155.0,167.84K,-6.15%


In [26]:
BXPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
1943,08/18/2014,39.0,39.0,40.0,39.0,1.03M,0.00%
1944,08/14/2014,39.0,39.0,39.0,39.0,797.00K,0.00%
1945,08/13/2014,39.0,39.0,39.0,38.0,285.77K,2.63%
1946,08/12/2014,38.0,39.0,40.0,38.0,587.19K,-2.56%
1947,08/11/2014,39.0,38.0,39.0,38.0,891.59K,2.63%


In [27]:
BXPH = BXPH.iloc[::-1]
BXPH.reset_index(drop=True, inplace=True)

In [28]:
BXPH.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,08/11/2014,39.0,38.0,39.0,38.0,891.59K,2.63%
1,08/12/2014,38.0,39.0,40.0,38.0,587.19K,-2.56%
2,08/13/2014,39.0,39.0,39.0,38.0,285.77K,2.63%
3,08/14/2014,39.0,39.0,39.0,39.0,797.00K,0.00%
4,08/18/2014,39.0,39.0,40.0,39.0,1.03M,0.00%


In [29]:
BXPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
1943,10/24/2022,160.2,170.0,170.0,155.0,167.84K,-6.15%
1944,10/25/2022,160.2,162.0,164.3,159.5,299.41K,0.00%
1945,10/26/2022,158.0,160.0,161.1,153.1,354.48K,-1.37%
1946,10/27/2022,156.4,159.0,160.9,156.0,185.86K,-1.01%
1947,10/30/2022,153.2,156.3,156.3,147.8,362.01K,-2.05%


In [30]:
BXPH.rename(columns={"Price":"Price_BXPH","Open":"Open_BXPH","High":"High_BXPH","Low":"Low_BXPH","Vol.":"Vol_BXPH","Change %":"Change_BXPH"},inplace=True)
BXPH.columns

Index(['Date', 'Price_BXPH', 'Open_BXPH', 'High_BXPH', 'Low_BXPH', 'Vol_BXPH',
       'Change_BXPH'],
      dtype='object')

In [31]:
BXPH.dtypes

Date            object
Price_BXPH     float64
Open_BXPH      float64
High_BXPH      float64
Low_BXPH       float64
Vol_BXPH        object
Change_BXPH     object
dtype: object

In [32]:
BXPH["Date"] = pd.to_datetime(BXPH["Date"])
BXPH.dtypes

Date           datetime64[ns]
Price_BXPH            float64
Open_BXPH             float64
High_BXPH             float64
Low_BXPH              float64
Vol_BXPH               object
Change_BXPH            object
dtype: object

In [33]:
print(BXPH.isna().sum().sum())
BXPH[BXPH.isna().any(axis=1)]

0


,Date,Price_BXPH,Open_BXPH,High_BXPH,Low_BXPH,Vol_BXPH,Change_BXPH


In [34]:
BXPH["Vol_BXPH"].bfill(limit=1, inplace=True)
print(BXPH.isna().sum().sum())
BXPH.reset_index(drop=True, inplace=True)

0


In [35]:
#Vol
BXPH["Vol_BXPH"] = BXPH["Vol_BXPH"].astype("str")
print(BXPH["Vol_BXPH"].apply(lambda x:x[-1:]).value_counts())
units = {'K':1e3, 'M':1e6}
BXPH["Vol_BXPH"] = BXPH["Vol_BXPH"].apply(lambda x:float(x[:-1])*units[x[-1:]])


#Change
BXPH["Change_BXPH"] = BXPH["Change_BXPH"].apply(lambda x:float(x[:-1]))

K    1247
M     701
Name: Vol_BXPH, dtype: int64


In [36]:
%%time

#calculating MACD
MACD(BXPH)

#calculating RSI
RSI(BXPH)

#calculating CCI
CCI(BXPH, number_of_days = 14)

#calculating RSI
ADX(BXPH)

CPU times: user 9.45 s, sys: 49.2 ms, total: 9.5 s
Wall time: 10 s


In [37]:
BXPH.drop(["Open_BXPH", "High_BXPH", "Low_BXPH", "Change_BXPH"], axis=1, inplace=True)
BXPH.head()

,Date,Price_BXPH,Vol_BXPH,MACD_BXPH,RSI_BXPH,CCI_BXPH,ADX_BXPH
0,2014-08-11,39.0,891590.0,0.000000,NaN,NaN,NaN
1,2014-08-12,38.0,587190.0,-0.079772,0.000000,NaN,NaN
2,2014-08-13,39.0,285770.0,-0.061590,7.142857,NaN,NaN
3,2014-08-14,39.0,797000.0,-0.046644,7.142857,NaN,NaN
4,2014-08-18,39.0,1030000.0,-0.034402,7.142857,NaN,NaN


In [38]:
BXPH.tail()

,Date,Price_BXPH,Vol_BXPH,MACD_BXPH,RSI_BXPH,CCI_BXPH,ADX_BXPH
1943,2022-10-24,160.2,167840.0,-1.428224,39.985917,-76.732074,22.02
1944,2022-10-25,160.2,299410.0,-1.849396,39.985917,-74.937598,22.04
1945,2022-10-26,158.0,354480.0,-2.333796,37.762082,-130.120408,22.05
1946,2022-10-27,156.4,185860.0,-2.814352,36.185864,-112.518223,22.53
1947,2022-10-30,153.2,362010.0,-3.414054,33.200990,-174.277146,22.98


In [39]:
BXPH.isna().sum()

Date           0
Price_BXPH     0
Vol_BXPH       0
MACD_BXPH      0
RSI_BXPH       1
CCI_BXPH      13
ADX_BXPH      27
dtype: int64

In [40]:
BXPH["RSI_BXPH"].bfill(limit=1, inplace=True)
BXPH["CCI_BXPH"].bfill(limit=13, inplace=True)
BXPH["ADX_BXPH"].bfill(limit=27, inplace=True)

In [41]:
BXPH.isna().sum()

Date          0
Price_BXPH    0
Vol_BXPH      0
MACD_BXPH     0
RSI_BXPH      0
CCI_BXPH      0
ADX_BXPH      0
dtype: int64

In [42]:
BXPH.dtypes

Date          datetime64[ns]
Price_BXPH           float64
Vol_BXPH             float64
MACD_BXPH            float64
RSI_BXPH             float64
CCI_BXPH             float64
ADX_BXPH             float64
dtype: object

In [43]:
BXPH.describe()

,Price_BXPH,Vol_BXPH,MACD_BXPH,RSI_BXPH,CCI_BXPH,ADX_BXPH
count,1948.000000,1.948000e+03,1948.000000,1948.000000,1948.000000,1948.000000
mean,105.472094,1.160276e+06,0.463081,50.727057,0.957014,23.604610
std,48.083122,1.439552e+06,2.948572,13.070103,110.803155,13.483102
min,38.000000,2.200000e+03,-6.169511,0.000000,-318.024691,5.420000
25%,71.887500,2.831200e+05,-1.095567,41.313388,-90.025273,13.465000
50%,86.000000,6.467700e+05,-0.033006,49.541563,-0.543095,19.970000
75%,115.350000,1.462500e+06,1.389566,59.717287,88.409605,30.475000
max,252.100000,1.402000e+07,16.343106,93.818506,466.666667,73.650000


# LBFL - LankaBangla Finance Ltd 

In [44]:
LBFL = pd.read_csv("/content/drive/MyDrive/stock project/data/LBFL.csv")
len_lbfl = len(LBFL)
print(len_lbfl)
LBFL.head()

3536


,Date,Price,Open,High,Low,Vol.,Change %
0,10/30/2022,26.0,26.0,26.0,26.0,4.43K,0.00%
1,10/27/2022,26.0,26.0,26.0,26.0,5.60K,0.00%
2,10/26/2022,26.0,26.0,26.0,26.0,5.55K,0.00%
3,10/25/2022,26.0,26.0,26.0,26.0,17.39K,0.00%
4,10/24/2022,26.0,26.0,26.0,26.0,0.41K,0.00%


In [45]:
LBFL.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3531,12/02/2007,7.73,7.96,7.96,7.55,4.08M,-2.28%
3532,11/29/2007,7.91,7.93,8.11,7.88,5.90M,0.64%
3533,11/28/2007,7.86,7.74,7.95,7.59,4.07M,1.16%
3534,11/27/2007,7.77,8.13,8.20,7.64,4.03M,-2.51%
3535,11/26/2007,7.97,7.51,8.13,7.51,8.68M,7.70%


In [46]:
LBFL = LBFL.iloc[::-1]
LBFL.reset_index(drop=True, inplace=True)
LBFL.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,11/26/2007,7.97,7.51,8.13,7.51,8.68M,7.70%
1,11/27/2007,7.77,8.13,8.20,7.64,4.03M,-2.51%
2,11/28/2007,7.86,7.74,7.95,7.59,4.07M,1.16%
3,11/29/2007,7.91,7.93,8.11,7.88,5.90M,0.64%
4,12/02/2007,7.73,7.96,7.96,7.55,4.08M,-2.28%


In [47]:
LBFL.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3531,10/24/2022,26.0,26.0,26.0,26.0,0.41K,0.00%
3532,10/25/2022,26.0,26.0,26.0,26.0,17.39K,0.00%
3533,10/26/2022,26.0,26.0,26.0,26.0,5.55K,0.00%
3534,10/27/2022,26.0,26.0,26.0,26.0,5.60K,0.00%
3535,10/30/2022,26.0,26.0,26.0,26.0,4.43K,0.00%


In [48]:
LBFL.rename(columns={"Price":"Price_LBFL","Open":"Open_LBFL","High":"High_LBFL","Low":"Low_LBFL","Vol.":"Vol_LBFL","Change %":"Change_LBFL"},inplace=True)
LBFL.columns

Index(['Date', 'Price_LBFL', 'Open_LBFL', 'High_LBFL', 'Low_LBFL', 'Vol_LBFL',
       'Change_LBFL'],
      dtype='object')

In [49]:
LBFL.dtypes

Date            object
Price_LBFL     float64
Open_LBFL      float64
High_LBFL      float64
Low_LBFL       float64
Vol_LBFL        object
Change_LBFL     object
dtype: object

In [50]:
LBFL["Date"] = pd.to_datetime(LBFL["Date"])
LBFL.dtypes

Date           datetime64[ns]
Price_LBFL            float64
Open_LBFL             float64
High_LBFL             float64
Low_LBFL              float64
Vol_LBFL               object
Change_LBFL            object
dtype: object

In [51]:
print(LBFL.isna().sum().sum())
LBFL[LBFL.isna().any(axis=1)]

2


,Date,Price_LBFL,Open_LBFL,High_LBFL,Low_LBFL,Vol_LBFL,Change_LBFL
87,2008-04-06,9.05,9.05,9.05,9.05,NaN,0.00%
2983,2020-07-16,13.40,13.40,13.40,13.40,NaN,5.79%


In [52]:
LBFL["Vol_LBFL"].bfill(limit=1, inplace=True)
print(LBFL.isna().sum().sum())
LBFL.reset_index(drop=True, inplace=True)

0


In [53]:
#Vol
LBFL["Vol_LBFL"] = LBFL["Vol_LBFL"].astype("str")
print(LBFL["Vol_LBFL"].apply(lambda x:x[-1:]).value_counts())
units = {'K':1e3, 'M':1e6}
LBFL["Vol_LBFL"] = LBFL["Vol_LBFL"].apply(lambda x:float(x[:-1])*units[x[-1:]])


#Change
LBFL["Change_LBFL"] = LBFL["Change_LBFL"].apply(lambda x:float(x[:-1]))

M    2663
K     873
Name: Vol_LBFL, dtype: int64


In [54]:
%%time

#calculating MACD
MACD(LBFL)

#calculating RSI
RSI(LBFL)

#calculating CCI
CCI(LBFL, number_of_days = 14)

#calculating RSI
ADX(LBFL)

CPU times: user 15.7 s, sys: 82.6 ms, total: 15.8 s
Wall time: 15.8 s


In [55]:
LBFL.drop(["Open_LBFL", "High_LBFL", "Low_LBFL", "Change_LBFL"], axis=1, inplace=True)
LBFL.head()

,Date,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL
0,2007-11-26,7.97,8680000.0,0.000000,NaN,NaN,NaN
1,2007-11-27,7.77,4030000.0,-0.015954,0.000000,NaN,NaN
2,2007-11-28,7.86,4070000.0,-0.021093,3.345725,NaN,NaN
3,2007-11-29,7.91,5900000.0,-0.020890,5.242501,NaN,NaN
4,2007-12-02,7.73,4080000.0,-0.034852,4.871841,NaN,NaN


In [56]:
LBFL.isna().sum()

Date           0
Price_LBFL     0
Vol_LBFL       0
MACD_LBFL      0
RSI_LBFL       1
CCI_LBFL      23
ADX_LBFL      27
dtype: int64

In [57]:
LBFL["RSI_LBFL"].bfill(limit=1, inplace=True)
LBFL["CCI_LBFL"].bfill(limit=19, inplace=True)
LBFL["ADX_LBFL"].bfill(limit=27, inplace=True)

In [58]:
LBFL.isna().sum()

Date           0
Price_LBFL     0
Vol_LBFL       0
MACD_LBFL      0
RSI_LBFL       0
CCI_LBFL      10
ADX_LBFL       0
dtype: int64

In [59]:
LBFL.head()

,Date,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL
0,2007-11-26,7.97,8680000.0,0.000000,0.000000,-1.831861,45.711429
1,2007-11-27,7.77,4030000.0,-0.015954,0.000000,-1.831861,45.711429
2,2007-11-28,7.86,4070000.0,-0.021093,3.345725,-1.831861,45.711429
3,2007-11-29,7.91,5900000.0,-0.020890,5.242501,-1.831861,45.711429
4,2007-12-02,7.73,4080000.0,-0.034852,4.871841,-1.831861,45.711429


In [60]:
LBFL.dtypes

Date          datetime64[ns]
Price_LBFL           float64
Vol_LBFL             float64
MACD_LBFL            float64
RSI_LBFL             float64
CCI_LBFL             float64
ADX_LBFL             float64
dtype: object

In [61]:
LBFL.describe()

,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL
count,3536.000000,3.536000e+03,3536.000000,3536.000000,3526.000000,3536.000000
mean,25.932825,3.937769e+06,0.036229,49.409235,-3.496662,21.478057
std,10.700892,4.459113e+06,1.008959,14.293609,111.524590,11.903899
min,6.980000,4.100000e+02,-3.098852,0.000000,-323.752678,4.730000
25%,17.035000,1.010000e+06,-0.565298,38.911689,-92.065443,12.760000
50%,24.000000,2.190000e+06,-0.041896,48.904577,-11.591560,18.180000
75%,33.080000,5.255000e+06,0.579007,58.992740,82.844687,27.315000
max,57.940000,3.734000e+07,5.438706,91.362310,466.666667,80.680000


In [62]:
LBFL[LBFL["CCI_LBFL"]==np.inf]

,Date,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL


In [63]:
LBFL.replace([np.inf, -np.inf], np.nan, inplace=True)
LBFL[LBFL["CCI_LBFL"]==np.inf]

,Date,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL


In [64]:
LBFL.isna().sum()

Date           0
Price_LBFL     0
Vol_LBFL       0
MACD_LBFL      0
RSI_LBFL       0
CCI_LBFL      10
ADX_LBFL       0
dtype: int64

In [65]:
LBFL["CCI_LBFL"].bfill(limit=1, inplace=True)
LBFL.isna().sum()

Date           0
Price_LBFL     0
Vol_LBFL       0
MACD_LBFL      0
RSI_LBFL       0
CCI_LBFL      10
ADX_LBFL       0
dtype: int64

In [66]:
LBFL.describe()

,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL
count,3536.000000,3.536000e+03,3536.000000,3536.000000,3526.000000,3536.000000
mean,25.932825,3.937769e+06,0.036229,49.409235,-3.496662,21.478057
std,10.700892,4.459113e+06,1.008959,14.293609,111.524590,11.903899
min,6.980000,4.100000e+02,-3.098852,0.000000,-323.752678,4.730000
25%,17.035000,1.010000e+06,-0.565298,38.911689,-92.065443,12.760000
50%,24.000000,2.190000e+06,-0.041896,48.904577,-11.591560,18.180000
75%,33.080000,5.255000e+06,0.579007,58.992740,82.844687,27.315000
max,57.940000,3.734000e+07,5.438706,91.362310,466.666667,80.680000


# RNTL - Renata Limited

In [67]:
RNTL = pd.read_csv("/content/drive/MyDrive/stock project/data/RNTL.csv")
len_rntl = len(RNTL)
print(len_rntl)
RNTL.head()

3529


,Date,Price,Open,High,Low,Vol.,Change %
0,10/30/2022,"1,303.20","1,303.20","1,303.20","1,303.20",0.93K,0.00%
1,10/27/2022,"1,303.20","1,303.20","1,303.20","1,303.20",0.71K,0.00%
2,10/26/2022,"1,303.20","1,303.20","1,303.20","1,303.20",0.44K,0.00%
3,10/25/2022,"1,303.20","1,303.20","1,303.20","1,303.20",0.95K,0.00%
4,10/24/2022,"1,303.20","1,303.20","1,303.20","1,303.20",0.91K,0.00%


In [68]:
RNTL['Price'] = RNTL['Price'].str.replace(',','').astype(np.float64)
RNTL['Open'] = RNTL['Open'].str.replace(',','').astype(np.float64)
RNTL['High'] = RNTL['High'].str.replace(',','').astype(np.float64)
RNTL['Low'] = RNTL['Low'].str.replace(',','').astype(np.float64)

In [69]:
RNTL.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,10/30/2022,1303.2,1303.2,1303.2,1303.2,0.93K,0.00%
1,10/27/2022,1303.2,1303.2,1303.2,1303.2,0.71K,0.00%
2,10/26/2022,1303.2,1303.2,1303.2,1303.2,0.44K,0.00%
3,10/25/2022,1303.2,1303.2,1303.2,1303.2,0.95K,0.00%
4,10/24/2022,1303.2,1303.2,1303.2,1303.2,0.91K,0.00%


In [70]:
RNTL.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3524,12/02/2007,67.31,66.26,68.99,66.26,33.36K,4.88%
3525,11/29/2007,64.18,63.13,64.73,63.12,51.89K,3.42%
3526,11/28/2007,62.06,60.43,63.65,60.43,21.31K,2.83%
3527,11/27/2007,60.35,60.43,60.43,59.89,17.14K,-0.12%
3528,11/26/2007,60.42,59.89,60.96,59.89,13.44K,2.11%


In [71]:
RNTL = RNTL.iloc[::-1]
RNTL.reset_index(drop=True, inplace=True)
RNTL.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,11/26/2007,60.42,59.89,60.96,59.89,13.44K,2.11%
1,11/27/2007,60.35,60.43,60.43,59.89,17.14K,-0.12%
2,11/28/2007,62.06,60.43,63.65,60.43,21.31K,2.83%
3,11/29/2007,64.18,63.13,64.73,63.12,51.89K,3.42%
4,12/02/2007,67.31,66.26,68.99,66.26,33.36K,4.88%


In [72]:
RNTL.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3524,10/24/2022,1303.2,1303.2,1303.2,1303.2,0.91K,0.00%
3525,10/25/2022,1303.2,1303.2,1303.2,1303.2,0.95K,0.00%
3526,10/26/2022,1303.2,1303.2,1303.2,1303.2,0.44K,0.00%
3527,10/27/2022,1303.2,1303.2,1303.2,1303.2,0.71K,0.00%
3528,10/30/2022,1303.2,1303.2,1303.2,1303.2,0.93K,0.00%


In [73]:
RNTL.rename(columns={"Price":"Price_RNTL","Open":"Open_RNTL","High":"High_RNTL","Low":"Low_RNTL","Vol.":"Vol_RNTL","Change %":"Change_RNTL"},inplace=True)
RNTL.columns

Index(['Date', 'Price_RNTL', 'Open_RNTL', 'High_RNTL', 'Low_RNTL', 'Vol_RNTL',
       'Change_RNTL'],
      dtype='object')

In [74]:
RNTL.dtypes

Date            object
Price_RNTL     float64
Open_RNTL      float64
High_RNTL      float64
Low_RNTL       float64
Vol_RNTL        object
Change_RNTL     object
dtype: object

In [75]:
RNTL["Date"] = pd.to_datetime(RNTL["Date"])
RNTL.dtypes

Date           datetime64[ns]
Price_RNTL            float64
Open_RNTL             float64
High_RNTL             float64
Low_RNTL              float64
Vol_RNTL               object
Change_RNTL            object
dtype: object

In [76]:
print(RNTL.isna().sum().sum())

1


In [77]:
RNTL["Vol_RNTL"].bfill(limit=1, inplace=True)
print(RNTL.isna().sum().sum())
RNTL.reset_index(drop=True, inplace=True)

0


In [78]:
#Vol
RNTL["Vol_RNTL"] = RNTL["Vol_RNTL"].astype("str")
print(RNTL["Vol_RNTL"].apply(lambda x:x[-1:]).value_counts())
units = {'K':1e3, 'M':1e6}
RNTL["Vol_RNTL"] = RNTL["Vol_RNTL"].apply(lambda x:float(x[:-1])*units[x[-1:]])


#Change
RNTL["Change_RNTL"] = RNTL["Change_RNTL"].apply(lambda x:float(x[:-1]))

K    3529
Name: Vol_RNTL, dtype: int64


In [79]:
%%time

#calculating MACD
MACD(RNTL)

#calculating RSI
RSI(RNTL)

#calculating CCI
CCI(RNTL, number_of_days = 14)

#calculating RSI
ADX(RNTL)

CPU times: user 15.6 s, sys: 116 ms, total: 15.7 s
Wall time: 15.7 s


In [80]:
RNTL.drop(["Open_RNTL", "High_RNTL", "Low_RNTL", "Change_RNTL"], axis=1, inplace=True)
RNTL.head()

,Date,Price_RNTL,Vol_RNTL,MACD_RNTL,RSI_RNTL,CCI_RNTL,ADX_RNTL
0,2007-11-26,60.42,13440.0,0.000000,NaN,NaN,NaN
1,2007-11-27,60.35,17140.0,-0.005584,0.000000,NaN,NaN
2,2007-11-28,62.06,21310.0,0.126515,65.267176,NaN,NaN
3,2007-11-29,64.18,51890.0,0.397686,81.440226,NaN,NaN
4,2007-12-02,67.31,33360.0,0.855297,89.335691,NaN,NaN


In [81]:
RNTL.isna().sum()

Date           0
Price_RNTL     0
Vol_RNTL       0
MACD_RNTL      0
RSI_RNTL       1
CCI_RNTL      13
ADX_RNTL      27
dtype: int64

In [82]:
RNTL["RSI_RNTL"].bfill(limit=1, inplace=True)
RNTL["CCI_RNTL"].bfill(limit=13, inplace=True)
RNTL["ADX_RNTL"].bfill(limit=27, inplace=True)

In [83]:
RNTL.isna().sum()

Date          0
Price_RNTL    0
Vol_RNTL      0
MACD_RNTL     0
RSI_RNTL      0
CCI_RNTL      0
ADX_RNTL      0
dtype: int64

In [84]:
RNTL.head()

,Date,Price_RNTL,Vol_RNTL,MACD_RNTL,RSI_RNTL,CCI_RNTL,ADX_RNTL
0,2007-11-26,60.42,13440.0,0.000000,0.000000,66.630535,8.528571
1,2007-11-27,60.35,17140.0,-0.005584,0.000000,66.630535,8.528571
2,2007-11-28,62.06,21310.0,0.126515,65.267176,66.630535,8.528571
3,2007-11-29,64.18,51890.0,0.397686,81.440226,66.630535,8.528571
4,2007-12-02,67.31,33360.0,0.855297,89.335691,66.630535,8.528571


In [85]:
RNTL.tail()

,Date,Price_RNTL,Vol_RNTL,MACD_RNTL,RSI_RNTL,CCI_RNTL,ADX_RNTL
3524,2022-10-24,1303.2,910.0,-0.503972,44.253417,0.000000,50.74
3525,2022-10-25,1303.2,950.0,-0.469588,44.253417,0.000000,51.10
3526,2022-10-26,1303.2,440.0,-0.437297,44.253417,0.000000,51.42
3527,2022-10-27,1303.2,710.0,-0.407014,44.253417,0.000000,51.72
3528,2022-10-30,1303.2,930.0,-0.378651,44.253417,-66.666667,52.00


In [86]:
RNTL.dtypes

Date          datetime64[ns]
Price_RNTL           float64
Vol_RNTL             float64
MACD_RNTL            float64
RSI_RNTL             float64
CCI_RNTL             float64
ADX_RNTL             float64
dtype: object

In [87]:
RNTL.describe()

,Price_RNTL,Vol_RNTL,MACD_RNTL,RSI_RNTL,CCI_RNTL,ADX_RNTL
count,3529.000000,3529.000000,3529.000000,3529.000000,3529.000000,3529.000000
mean,545.193772,26022.567300,2.466513,53.848664,16.852655,25.364605
std,386.590471,39793.057294,6.544564,11.814622,109.292325,11.400787
min,60.350000,240.000000,-23.545630,0.000000,-430.303451,5.360000
25%,205.140000,7450.000000,-1.214082,45.787817,-65.954837,16.410000
50%,469.550000,14250.000000,1.012209,52.980154,21.935523,23.090000
75%,848.100000,28470.000000,5.194998,61.104588,89.036646,33.420000
max,1378.300000,734660.000000,40.688184,94.064472,466.666667,59.190000


# Merging - 3 Stocks

In [88]:
print(min(len_bxph, len_lbfl, len_rntl))
print(max(len_bxph, len_lbfl, len_rntl))

1948
3536


In [89]:
data = pd.merge(pd.merge(BXPH,LBFL,on='Date'),RNTL,on='Date')
print(len(data))
data.head()

1929


,Date,Price_BXPH,Vol_BXPH,MACD_BXPH,RSI_BXPH,CCI_BXPH,ADX_BXPH,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL,Price_RNTL,Vol_RNTL,MACD_RNTL,RSI_RNTL,CCI_RNTL,ADX_RNTL
0,2014-08-11,39.0,891590.0,0.000000,0.000000,112.982456,52.812857,20.20,3340000.0,0.053881,62.717594,235.249316,33.10,372.77,48950.0,3.842148,55.568779,-9.681690,38.72
1,2014-08-12,38.0,587190.0,-0.079772,0.000000,112.982456,52.812857,19.47,2340000.0,0.067975,52.885274,95.745885,32.77,371.57,50400.0,3.514794,54.047380,-104.349478,38.82
2,2014-08-13,39.0,285770.0,-0.061590,7.142857,112.982456,52.812857,19.38,1560000.0,0.071063,51.806926,53.936348,32.45,371.94,21980.0,3.247782,54.461377,-70.225619,39.12
3,2014-08-14,39.0,797000.0,-0.046644,7.142857,112.982456,52.812857,19.61,1190000.0,0.091020,54.367673,51.645704,32.16,371.74,49550.0,2.985618,54.177248,-96.913645,39.26
4,2014-08-18,39.0,1030000.0,-0.034402,7.142857,112.982456,52.812857,19.33,1390000.0,0.083283,50.826957,44.253440,32.17,368.03,40320.0,2.450240,49.063757,-223.266795,39.39


In [90]:
data.tail()

,Date,Price_BXPH,Vol_BXPH,MACD_BXPH,RSI_BXPH,CCI_BXPH,ADX_BXPH,Price_LBFL,Vol_LBFL,MACD_LBFL,RSI_LBFL,CCI_LBFL,ADX_LBFL,Price_RNTL,Vol_RNTL,MACD_RNTL,RSI_RNTL,CCI_RNTL,ADX_RNTL
1924,2022-10-24,160.2,167840.0,-1.428224,39.985917,-76.732074,22.02,26.0,410.0,-0.190909,38.659313,NaN,29.55,1303.2,910.0,-0.503972,44.253417,0.000000,50.74
1925,2022-10-25,160.2,299410.0,-1.849396,39.985917,-74.937598,22.04,26.0,17390.0,-0.178547,38.659313,NaN,29.46,1303.2,950.0,-0.469588,44.253417,0.000000,51.10
1926,2022-10-26,158.0,354480.0,-2.333796,37.762082,-130.120408,22.05,26.0,5550.0,-0.166827,38.659313,NaN,29.36,1303.2,440.0,-0.437297,44.253417,0.000000,51.42
1927,2022-10-27,156.4,185860.0,-2.814352,36.185864,-112.518223,22.53,26.0,5600.0,-0.155743,38.659313,NaN,29.24,1303.2,710.0,-0.407014,44.253417,0.000000,51.72
1928,2022-10-30,153.2,362010.0,-3.414054,33.200990,-174.277146,22.98,26.0,4430.0,-0.145284,38.659313,NaN,29.10,1303.2,930.0,-0.378651,44.253417,-66.666667,52.00


In [91]:
data.columns

Index(['Date', 'Price_BXPH', 'Vol_BXPH', 'MACD_BXPH', 'RSI_BXPH', 'CCI_BXPH',
       'ADX_BXPH', 'Price_LBFL', 'Vol_LBFL', 'MACD_LBFL', 'RSI_LBFL',
       'CCI_LBFL', 'ADX_LBFL', 'Price_RNTL', 'Vol_RNTL', 'MACD_RNTL',
       'RSI_RNTL', 'CCI_RNTL', 'ADX_RNTL'],
      dtype='object')

In [92]:
data.isna().sum().sum()

10

In [94]:
data.to_csv("/content/drive/MyDrive/stock project/data/stock_data.csv",index=False)